# 9. Error handling

Use `Result<T, E>` for recoverable errors

Use `panic!` macro for unrecoverable errors

## `panic!`

By dafault, rust 


In [2]:
{
  panic!("crush!")
}

thread '<unnamed>' panicked at 'crush!', src/lib.rs:22:3
stack backtrace:
   0: std::panicking::begin_panic
   1: _run_user_code_1
   2: evcxr::runtime::Runtime::run_loop
   3: evcxr::runtime::runtime_hook
   4: evcxr_jupyter::main
note: Some details are omitted, run with `RUST_BACKTRACE=full` for a verbose backtrace.
Segmentation fault.
   0: backtrace::backtrace::trace
   1: backtrace::capture::Backtrace::new
   2: evcxr::runtime::Runtime::install_crash_handlers::segfault_handler
   3: __sigtramp
   4: core::ptr::drop_in_place
             at /rustc/7eac88abb2e57e752f3302f02be5f3ce3d7adfb4/library/core/src/ptr/mod.rs:175:1
      core::ptr::drop_in_place
             at /rustc/7eac88abb2e57e752f3302f02be5f3ce3d7adfb4/library/core/src/ptr/mod.rs:175:1
      core::result::Result<T,E>::unwrap_or
             at /rustc/7eac88abb2e57e752f3302f02be5f3ce3d7adfb4/library/core/src/result.rs:805:5
      std::rt::lang_start_internal
             at /rustc/7eac88abb2e57e752f3302f02be5f3ce3d7adfb4

Error: Child process terminated with status: signal: 6

In [3]:
let v = vec![1,2,3];
v[99];

thread '<unnamed>' panicked at 'index out of bounds: the len is 3 but the index is 99', src/lib.rs:106:1
stack backtrace:
   0: _rust_begin_unwind
   1: core::panicking::panic_fmt
   2: core::panicking::panic_bounds_check
   3: _run_user_code_1
   4: evcxr::runtime::Runtime::run_loop
   5: evcxr::runtime::runtime_hook
   6: evcxr_jupyter::main
note: Some details are omitted, run with `RUST_BACKTRACE=full` for a verbose backtrace.


Error: Child process terminated with status: exit code: 101

When we set `RUST_BACKTRACE=1`, we can get detailed reason.

## Result
Use `Result` to handle error. 
`Result` enum has `Ok` and `Err`. We can write `match` expression and control each pattern.

In [3]:
use std::fs::File;

let f = File::open("hello.txt");

let f = match f {
  Ok(file) => file,
  Err(error) => {
    panic!("There was a problem opening the file: {:?}", error)
  },
};

thread '<unnamed>' panicked at 'There was a problem opening the file: Os { code: 2, kind: NotFound, message: "No such file or directory" }', src/lib.rs:111:5
stack backtrace:
   0: _rust_begin_unwind
   1: std::panicking::begin_panic_fmt
   2: _run_user_code_1
   3: evcxr::runtime::Runtime::run_loop
   4: evcxr::runtime::runtime_hook
   5: evcxr_jupyter::main
note: Some details are omitted, run with `RUST_BACKTRACE=full` for a verbose backtrace.


Error: Child process terminated with status: exit code: 101

We can change error handling by error type.
In this case, we'll create new file if file were not found, invoke `panic!` if file could not be opened.

In [3]:
use std::fs::File;
use std::io::ErrorKind;

let f = File::open("ss09/hello.txt");

let f = match f {
  Ok(file) => file,
  Err(ref error) if error.kind() == ErrorKind::NotFound => {
    match File::create("ss09/hello.txt") {
      Ok(fc) => fc,
      Err(e) => {
        panic!("Tried to create file but there was a problem: {:?}", e)
      },
    }
  },
  Err(error) => {
    panic!("There was a problem opening the file: {:?}", error)
  },
};

println!("{:?}", f);

File { fd: 3, path: "/Users/ta.nakamura/workspace/na9amura/tutorials/rust/ss09/hello.txt", read: false, write: true }


When we don't need to have complex error handling, using `unwrap` or `ecxpect` is good enough. Sometimes `match` statement is too long to handle simple error. 
- `Result#unrwap` returns contents of `Ok` or call `panic`.
- `Result#expect` returns contents of `Ok` or call `panic` with indicated message.

In [4]:
let f = File::open("hello.txt").unwrap();

thread '<unnamed>' panicked at 'called `Result::unwrap()` on an `Err` value: Os { code: 2, kind: NotFound, message: "No such file or directory" }', src/lib.rs:109:33
stack backtrace:
   0: _rust_begin_unwind
   1: core::panicking::panic_fmt
   2: core::option::expect_none_failed
   3: _run_user_code_2
   4: evcxr::runtime::Runtime::run_loop
   5: evcxr::runtime::runtime_hook
   6: evcxr_jupyter::main
note: Some details are omitted, run with `RUST_BACKTRACE=full` for a verbose backtrace.


Error: Child process terminated with status: exit code: 101

In [5]:
let f = File::open("hello.txt").expect("Failed to open file");

thread '<unnamed>' panicked at 'Failed to open file: Os { code: 2, kind: NotFound, message: "No such file or directory" }', src/lib.rs:107:33
stack backtrace:
   0: _rust_begin_unwind
   1: core::panicking::panic_fmt
   2: core::option::expect_none_failed
   3: _run_user_code_2
   4: evcxr::runtime::Runtime::run_loop
   5: evcxr::runtime::runtime_hook
   6: evcxr_jupyter::main
note: Some details are omitted, run with `RUST_BACKTRACE=full` for a verbose backtrace.


Error: Child process terminated with status: exit code: 101

## Delegate error
When we define function, we usually delegate error handling to callee.
It makes us to be able to write error handler in appropriate block.

In [26]:
use std::io;
use std::io::Read;
use std::fs::File;

fn read_username_from_file (path: &str) -> Result<String, io::Error> {
  let f = File::open(path);

  let mut f = match f {
    Ok(file) => file,
    Err(e) => return Err(e),
  };

  let mut s = String::new(); // read_to_string will bind contents of file here
  match f.read_to_string(&mut s) {
    Ok(_) => Ok(s),
    Err(e) => Err(e),
  }
}

In this case, `read_username_from_file` is be able to concentrate to read file.
We need to handle error when we call this function.

In [33]:
let s = match read_username_from_file("ss09/hello.txt") {
  Ok(s) => s,
  Err(_) => String::from("guest"),
};

println!("Hello, {}", s);

Hello, na9amura


In [34]:
let s = read_username_from_file("ss09/foobar.txt").unwrap_or(String::from("guest"));

println!("Hello, {}", s);

Hello, guest


### Error delegation in shorthand
We can write `?` operator to delegete error handling in short style.

In [35]:
use std::io;
use std::io::Read;
use std::fs::File;

fn read_username_from_file (path: &str) -> Result<String, io::Error> {
  let mut f = File::open("hello.txt")?; // returns contents of Err here if error occured.
  let mut s = String::new();
  f.read_to_string(&mut s)?; // returns contents of Err here if error occured.
  Ok(s)
}

`?` operator is really simillar to `match` statement. But different from it, `?` operator pass the error to `from` function in `From` trait.
(Note: `From::from` converts error by type inference? In this case, `From::from` converts error to `io::Error`?)

We can chain methods using `?` operator.

In [ ]:
use std::io;
use std::io::Read;
use std::fs::File;

fn read_username_from_file (path: &str) -> Result<String, io::Error> {
  let mut s = String::new();
  File::open("hello.txt")?.read_to_string(&mut s)?;
  Ok(s)
}

### `?` operator must be used in function returns `Result`
(Note: Currently we can use in function returns `Result` or `Optioin`)

In [37]:
fn read () {
  let f = File::open("hello.txt")?;
}

Error: the `?` operator can only be used in a function that returns `Result` or `Option` (or another type that implements `Try`)

In [11]:
use std::io;
use std::fs::File;

fn read () -> Result<File, std::io::Error> {
  let f = File::open("hello.txt")?;
  Ok(f)
}

## panic! or Result

### prefer panic
- examples, protprype, test code
- 


In [15]:
use std::net::IpAddr;

"127.0.0.1".parse::<IpAddr>()

Ok(127.0.0.1)

In [17]:
let home: IpAddr = "127.0.0.1".parse().unwrap();
home

127.0.0.1

↑ これは絶対失敗しないのでエラー処理なしに unwrap して問題ない。

> 悪い状態とは、何らかの前提、保証、契約、不変性が破られたこと

- 以下のような値がプログラムに渡される
  - 無効な値
  - 矛盾する値
  - 行方不明な値

- かつ
  - エラー処理が正しく行われていない
  - 

### 検証のために独自の型を作る

In [21]:
let guess = String::from("100");
let guess: u32 = match guess.trim().parse() {
  Ok(num) => num,
  Err(_) => {
    println!("failed to parse");
    0
  },
};

In [23]:
let guess = String::from("-100");
let guess: i32 = match guess.trim().parse() {
  Ok(num) => num,
  Err(_) => {
    println!("failed to parse");
    -1 // tmp
  },
};

if guess < 1 || guess > 100 {
  println!("The secret number will be between 1 and 100");
}

The secret number will be between 1 and 100


()

> 新しい型を作って検証を関数内に閉じ込め、検証を全箇所で繰り返すのではなく、 その型のインスタンスを生成する

In [25]:
pub struct Guess {
  value: u32,
}

impl Guess {
  pub fn new(value: u32) -> Guess {
    if value < 1 || value > 100 {
      panic!("Guess value must be between 1 and 100, got {}.", value);
    }

    Guess {
      value
    }
  }

  pub fn value(&self) -> u32 {
    self.value
  }
}